<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-openai-agents/blob/main/05_mcp/main_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 5.1

### Take Aways
* Open
* Open
* Open


https://platform.openai.com/logs?api=traces

https://www.youtube.com/playlist?list=PLO66QfE8gWT0oM1hbfcFUa-2H3yI4vfg8

In [ ]:
# Install required packages

!pip install openai-agents
!pip install mcp[cli]
!pip install fastmcp nest_asyncio

In [ ]:
import os
import warnings
from google.colab import userdata

# Filter specific DeprecationWarning from jupyter_client
warnings.filterwarnings("ignore", category=DeprecationWarning, module='jupyter_client')

# Initialize API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
# set up imports

import requests
import nest_asyncio
import threading
import time
from fastmcp import FastMCP
from typing import Any
import httpx
import warnings


# Filter specific DeprecationWarning from jupyter_client
warnings.filterwarnings("ignore", category=DeprecationWarning, module='jupyter_client')

nest_asyncio.apply()

# create server
mcp = FastMCP("Weather Server")

@mcp.tool()
def get_weather(city : str)-> str :
    endpoint = "https://wttr.in"
    response = requests.get(f"{endpoint}/{city}")
    return response.text


@mcp.tool()
def add_numbers(a : int, b : int) -> int:
    return a + b

# Define the function to run the server
def run_server():
    # Use transport="streamable-http" for compatibility with notebooks/Colab
    print("🚀 Starting FastMCP server in background thread...")
    mcp.run(transport="streamable-http", host="0.0.0.0", port=8000)

# Start the server in a separate thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Give the server a moment to start up
time.sleep(5)
print("✅ Server should be running. Access it at http://localhost:8000/mcp")

In [ ]:
# Run some tests on the MCP server.

import asyncio
import warnings
from fastmcp import Client

# Filter specific DeprecationWarning from jupyter_client
warnings.filterwarnings("ignore", category=DeprecationWarning, module='jupyter_client')

client = Client("http://localhost:8000/mcp")

async def call_get_weather(name: str):
    async with client:
        result = await client.call_tool("get_weather", {"city": name})
        print(result)


async def call_add_numbers(a: int, b: int):
    async with client:
        result = await client.call_tool("add_numbers", {"a": a, "b": b})
        print(result)


async def call_list_tools():
    async with client:
        result = await client.list_tools()
        for tool in result:
            print(tool)

asyncio.run(call_get_weather("Seattle"))
asyncio.run(call_add_numbers(2, 4))
asyncio.run(call_list_tools())